# Milei-GPT Inference
- www.machinelearnear.com

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers trl peft accelerate bitsandbytes
!pip install --upgrade gradio

In [ ]:
import os
os.environ['HF_HOME'] = 'YOUR_CACHE_DIR'

## Ejecutar en la Notebook

In [ ]:
# from unsloth import FastLanguageModel
# import torch
# max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
# dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
# load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# # load model
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = "machinelearnear/llama-3-8b-milei-gpt",
#     # model_name = "machinelearnear/Phi-3-medium-4k-instruct-milei-gpt",
#     # model_name = "machinelearnear/mistral-7b-instruct-v0.3-bnb-4bit-milei-gpt",
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
#     # token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
# )

# FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [ ]:
# # function to read yaml file and get system message
# def read_yaml(config_path):
#     import yaml
#     with open(config_path, 'r') as file:
#         return yaml.safe_load(file)
    
# config = read_yaml("../scripts/unsloth_finetuning_config.yaml")
# system_message = config.get('system_message', 'You are a nice bot!')

In [ ]:
# messages = [
#     {
#         "content": system_message,
#         "role": "system",
#     },
#     {
#         "content": "Che te comerias un asado con unos politicos?",
#         "role": "user"
#     },
# ]

# inputs = tokenizer.apply_chat_template(
#     messages,
#     tokenize = True,
#     add_generation_prompt = True, # Must add for generation
#     return_tensors = "pt",
# ).to("cuda")

In [ ]:
# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(
#     input_ids = inputs,
#     streamer = text_streamer,
#     max_new_tokens = 1000,
#     use_cache = True,
#     temperature=0.1, # creativity and randomness of the response
#     top_p=0.9, # dynamically adjusts the number of choices for each predicted token, which helps to maintain diversity and generate more fluent and natural-sounding text
#     top_k=50, # limits the number of choices for the next predicted word or token, which helps to speed up the generation process and can improve the quality of the generated text
#     repetition_penalty=1.2, # reduce the likelihood of repeating prompt text or getting stuck in a loop
# )

## Ejecutar dentro de una app de Gradio

In [1]:
from threading import Thread
from typing import Iterator

import gradio as gr
import torch
from unsloth import FastLanguageModel
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
import time

MAX_MAX_NEW_TOKENS = 2048
DEFAULT_MAX_NEW_TOKENS = 1000
MAX_INPUT_TOKEN_LENGTH = 4096
DTYPE = None
LOAD_IN_4BIT = True

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [2]:
import os
import requests
import yaml

CONFIG_PATH = "../scripts/unsloth_finetuning_config.yaml"
if not os.path.exists(CONFIG_PATH):
	response = requests.get("https://raw.githubusercontent.com/machinelearnear/milei-gpt/main/scripts/unsloth_finetuning_config.yaml")
	config = yaml.safe_load(response.content)
else:
	config = yaml.safe_load(CONFIG_PATH)

system_prompt = config.get('system_message', 'You are a nice bot!')

In [ ]:
PLACEHOLDER = """
<div style="padding: 50px; text-align: center; display: flex; flex-direction: column; align-items: center;">
   <img src="https://avatars.githubusercontent.com/u/78419164?v=4" style="width: 80%; max-width: 550px; height: auto; opacity: 0.80;  "> 
   <h1 style="font-size: 36px; margin-bottom: 15px; opacity: 0.85; font-family: 'Source Sans Pro', sans-serif;">Llama-3-8B-Instruct-Milei-GPT</h1>
</div>
"""

custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Source+Sans+Pro:wght@300;400;700&display=swap');

body, .gradio-container, .gr-button, .gr-input, .gr-slider, .gr-dropdown, .gr-markdown {
    font-family: 'Source Sans Pro', sans-serif !important;
    margin: 0;
    padding: 0;
    box-sizing: border-box;
    font-size: 18px;
}
._button {
    font-size: 20px;
    padding: 12px 24px;
}
pre, code {
    direction: ltr !important;
    unicode-bidi: plaintext !important;
    font-size: 16px;
}
"""

In [ ]:
def execution_time_calculator(start_time, log=True):
    delta = time.time() - start_time
    if log:
        print("--- %s seconds ---" % (delta))
    return delta

def token_per_second_calculator(tokens_count, time_delta):
    return tokens_count/time_delta

if not torch.cuda.is_available():
    DESCRIPTION = "\n<p>Running on CPU 🥶 This demo does not work on CPU.</p>"

if torch.cuda.is_available():
    # Load model and tokenizer
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="machinelearnear/llama-3-8b-milei-gpt",
        max_seq_length=MAX_MAX_NEW_TOKENS,
        dtype=DTYPE,
        load_in_4bit=LOAD_IN_4BIT,
    )
    FastLanguageModel.for_inference(model)
    
generation_speed = 0

def get_generation_speed():
    global generation_speed
    return generation_speed

def generate(
    message: str,
    chat_history: list[tuple[str, str]],
    max_new_tokens: int = 1000,
    temperature: float = 0.7,
    top_p: float = 0.9,
    top_k: int = 30,
    repetition_penalty: float = 1.2,
    do_sample: bool =True,
) -> Iterator[str]:
    global generation_speed
    global system_prompt

    conversation = []
    if system_prompt:
        conversation.append({"role": "system", "content": system_prompt})
    for user, assistant in chat_history:
        conversation.extend([{"role": "user", "content": user}, {"role": "assistant", "content": assistant}])
    conversation.append({"role": "user", "content": message})

    input_ids = tokenizer.apply_chat_template(conversation, return_tensors="pt")
    if input_ids.shape[1] > MAX_INPUT_TOKEN_LENGTH:
        input_ids = input_ids[:, -MAX_INPUT_TOKEN_LENGTH:]
        gr.Warning(f"Trimmed input from conversation as it was longer than {MAX_INPUT_TOKEN_LENGTH} tokens.")
    input_ids = input_ids.to(model.device)

    streamer = TextIteratorStreamer(tokenizer, timeout=10.0, skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        {"input_ids": input_ids},
        streamer=streamer,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        top_p=top_p,
        top_k=top_k,
        temperature=temperature,
        num_beams=1, # streamer only works with num_beams=1
        repetition_penalty=repetition_penalty,
    )

    start_time = time.time()
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()

    outputs = []
    sum_tokens = 0
    for text in streamer:
        num_tokens = len(tokenizer.tokenize(text))
        sum_tokens += num_tokens
        
        outputs.append(text)
        yield "".join(outputs)

    time_delta = execution_time_calculator(start_time, log=False)

    generation_speed = token_per_second_calculator(sum_tokens, time_delta)

    print(f"generation_speed: {generation_speed} tk/s")


chatbot = gr.Chatbot(placeholder=PLACEHOLDER, scale=1, show_copy_button=True, height="68%", rtl=False,  elem_classes=["chatbot"])
chat_input = gr.Textbox(show_label=True, rtl=False, placeholder="Input", show_copy_button=True, scale=5)
submit_btn = gr.Button(variant="primary", value="Submit", size="sm", scale=1, elem_classes=["_button"])


chat_interface = gr.ChatInterface(
    fn=generate,
    additional_inputs_accordion=gr.Accordion(label="Configuration", open=False),
    additional_inputs=[
        gr.Slider(
            label="Maximum number of tokens",
            minimum=1,
            maximum=MAX_MAX_NEW_TOKENS,
            step=1,
            value=DEFAULT_MAX_NEW_TOKENS,
        ),
        gr.Slider(
            label="Temperature",
            minimum=0.01,
            maximum=4.0,
            step=0.01,
            value=0.1,
        ),
        gr.Slider(
            label="Top-p",
            minimum=0.05,
            maximum=1.0,
            step=0.01,
            value=0.9,
        ),
        gr.Slider(
            label="Top-k",
            minimum=1,
            maximum=1000,
            step=1,
            value=30,
        ),
        gr.Slider(
            label="Repetition penalty",
            minimum=1.0,
            maximum=2.0,
            step=0.05,
            value=1.20,
        ),
        gr.Dropdown(
            label="Sampling",
            choices=[False, True],
            value=True)
    ],
    stop_btn="Stop",
    chatbot=chatbot,
    textbox=chat_input,
    submit_btn=submit_btn,
    retry_btn="🔄 Retry",
    undo_btn="↩️ Undo",
    clear_btn="🗑️ Clear",
    title="Llama-3-8B-Instruct-Milei-GPT Chatbot",
    description="Hecho con ❤️ y 🧉 por @machinelearnear",
    examples=[
        ["Te irías a comer un asado con alguien de la casta política?"],
        ["Che y cuantos perros tenes realmente? dame los nombres"],
        ["Las calles deberian ser privadas o públicas?"],
    ],
)


with gr.Blocks(css=custom_css, fill_height=False) as demo:
    # gr.Markdown(DESCRIPTION)
    chat_interface.render()

In [ ]:
demo.queue(max_size=20).launch(debug=True, share=True, inline=False)